Let's play a bit with geopandas and output the shape files for the WRA's water level data.

In [1]:
import numpy as np
import pandas as pd

import datetime
date = datetime.datetime.now().strftime('%Y%m%d')

In [8]:
zaf_df = pd.read_csv('data/database_ZAF_wl_clean_20211008.csv', 
    dtype={'井名': object, '井號': object, '日期時間': int,	'水位(m)': float}, 
    parse_dates=['日期時間'])

In [10]:
zaf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38697849 entries, 0 to 38697848
Data columns (total 4 columns):
 #   Column  Dtype         
---  ------  -----         
 0   井名      object        
 1   井號      object        
 2   日期時間    datetime64[ns]
 3   水位(m)   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.2+ GB


# Combine GPS

In [11]:
# try multiprocess
from dask import delayed
from dask.distributed import Client
import dask.dataframe as dd
client = Client(memory_limit='25GB')

In [3]:
excel_df = pd.read_excel('data/環保署水利署地下水井.xlsx')
excel_df.head()

,SiteId,SiteName,SiteEngName,County,Township,UgwDistName,TWD97Lon,TWD97Lat,TWD97TM2X,TWD97TM2Y,SiteAddress,StatusOfUse,數據紀錄始,數據紀錄終,資料數,data
0,4035,二林營運所,Erling Operation Center,彰化縣,二林鎮,濁水溪沖積扇,NaN,NaN,NaN,NaN,彰化縣二林鎮儒林路二段100號,廢站,1993/04,1999/05,18.0,環保署
1,4413,二崙國小,ErLun Elementary School,雲林縣,二崙鄉,濁水溪沖積扇,120.415102,23.771842,190386.40,2629878.33,雲林縣二崙鄉崙東村中興路36號,啟用,1999/08,2020/10,81.0,環保署
2,4133,二崙國小,ErLun Elementary School,雲林縣,二崙鄉,濁水溪沖積扇,NaN,NaN,NaN,NaN,雲林縣二崙鄉崙東村中興路36號,廢站,NaN,NaN,NaN,環保署
3,4403,口湖國小青蚶分校,"Kaohu Elementary School, Chingan Subdivision",雲林縣,口湖鄉,濁水溪沖積扇,120.161008,23.597331,164374.03,2610680.84,雲林縣口湖鄉青蚶村93號,啟用,1999/12,2020/10,84.0,環保署
4,4139,口湖國小青蚶分校,"Kaohu Elementary School, Chingan Subdivision",雲林縣,口湖鄉,濁水溪沖積扇,NaN,NaN,NaN,NaN,雲林縣口湖鄉青蚶村93號,廢站,NaN,NaN,NaN,環保署


In [12]:
def add_gps(well_id, df, excel):
    X = df[df['井號'] == well_id].copy()
    X['Lon'] = np.repeat(excel.loc[excel.SiteId.astype(str) == well_id, 'TWD97Lon'].values, len(X))
    X['Lat'] = np.repeat(excel.loc[excel.SiteId.astype(str) == well_id, 'TWD97Lat'].values, len(X))
    return X
def concat(compile_list):
    return pd.concat(compile_list, axis=0, join='outer')

df_future = client.scatter(zaf_df)
#zaf_dd = dd.read_csv('data/database_ZAF_wl_clean_20211008.csv')
compile_list = []
for well_id in zaf_df['井號'].unique():
    df = delayed(add_gps)(well_id, df_future, excel_df)
    compile_list.append(df)
    
total = delayed(concat)(compile_list)

In [13]:
total.visualize('results/add_gps.svg')

In [14]:
# computation costs only 248s
zaf_mo = total.compute()

In [15]:
zaf_mo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38697849 entries, 0 to 38388795
Data columns (total 6 columns):
 #   Column  Dtype         
---  ------  -----         
 0   井名      object        
 1   井號      object        
 2   日期時間    datetime64[ns]
 3   水位(m)   float64       
 4   Lon     float64       
 5   Lat     float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 2.0+ GB


In [16]:
zaf_mo.to_hdf('data/database_ZAF_clean_gps_{}.hd5'.format(date), key='wl', mode='w', encoding='utf-8')

In [23]:
zaf_mo.to_csv('data/database_ZAF_wl_clean_gps_{}.csv'.format(date), encoding='utf-8', index=False)

In [17]:
client.close()

# Play Geopandas

In [22]:
zaf_mo = pd.read_hdf('data/database_ZAF_clean_gps_20211008.hd5', key='wl')

In [3]:
zaf_mo.head()

,井名,井號,日期時間,水位(m),Lon,Lat
0,顏厝,070215T1,2007-01-24,0.48,120.434505,24.074449
1,顏厝,070215T1,2007-01-26,0.34,120.434505,24.074449
2,顏厝,070215T1,2007-01-27,0.71,120.434505,24.074449
3,顏厝,070215T1,2007-01-28,0.67,120.434505,24.074449
4,顏厝,070215T1,2007-01-29,0.16,120.434505,24.074449


In [4]:
zaf_mo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38697849 entries, 0 to 38388795
Data columns (total 6 columns):
 #   Column  Dtype         
---  ------  -----         
 0   井名      object        
 1   井號      object        
 2   日期時間    datetime64[ns]
 3   水位(m)   float64       
 4   Lon     float64       
 5   Lat     float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 2.0+ GB


In [6]:
zaf_mo['日期時間'].astype(str)

0           2007-01-24 00:00:00
1           2007-01-26 00:00:00
2           2007-01-27 00:00:00
3           2007-01-28 00:00:00
4           2007-01-29 00:00:00
                   ...         
38388791    2021-09-21 23:10:00
38388792    2021-09-21 23:20:00
38388793    2021-09-21 23:30:00
38388794    2021-09-21 23:40:00
38388795    2021-09-21 23:50:00
Name: 日期時間, Length: 38697849, dtype: object

In [8]:
zaf_mo['日期時間'] = zaf_mo['日期時間'].astype(str)

In [9]:
import geopandas as gd
gdf = gd.GeoDataFrame(
    zaf_mo, geometry=gd.points_from_xy(zaf_mo.Lon, zaf_mo.Lat))

In [10]:
print(gdf.head())

   井名        井號                 日期時間  水位(m)         Lon        Lat  \
0  顏厝  070215T1  2007-01-24 00:00:00   0.48  120.434505  24.074449   
1  顏厝  070215T1  2007-01-26 00:00:00   0.34  120.434505  24.074449   
2  顏厝  070215T1  2007-01-27 00:00:00   0.71  120.434505  24.074449   
3  顏厝  070215T1  2007-01-28 00:00:00   0.67  120.434505  24.074449   
4  顏厝  070215T1  2007-01-29 00:00:00   0.16  120.434505  24.074449   

                     geometry  
0  POINT (120.43451 24.07445)  
1  POINT (120.43451 24.07445)  
2  POINT (120.43451 24.07445)  
3  POINT (120.43451 24.07445)  
4  POINT (120.43451 24.07445)  


In [11]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 38697849 entries, 0 to 38388795
Data columns (total 7 columns):
 #   Column    Dtype   
---  ------    -----   
 0   井名        object  
 1   井號        object  
 2   日期時間      object  
 3   水位(m)     float64 
 4   Lon       float64 
 5   Lat       float64 
 6   geometry  geometry
dtypes: float64(3), geometry(1), object(3)
memory usage: 2.3+ GB


In [18]:
# exporting shape
gdf.columns = ['SiteName', 'SiteId', 'Timedate', 'WaterLv_m', 'Lon', 'Lat', 'geometry']

## Export shape files

In [19]:
# it cost 7411s...
gdf.to_file('data/ZAF_wl_wra_{}.shp'.format(date))